In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/beyond-analysis/train.csv
/kaggle/input/beyond-analysis/sample_submission_random.csv
/kaggle/input/beyond-analysis/test.csv


In [2]:
data=pd.read_csv("/kaggle/input/beyond-analysis/train.csv")
data.head()

,UNIQUE_IDENTIFIER,SEQUENCE_NO,STATUS_CHECK,CATEGORY_1,CATEGORY_2,ACTIVE_YN,ENTRY,REVENUE,WINNINGS_1,WINNINGS_2,...,WITHDRAW_NUMBER,DEPOSIT_TRAILS,ENTRY_NUMBER,WINNINGS_NUMBER,PRACTICE_ENTRY,PRACTICE_WINNINGS,PRACTICE_ENTRY_NUMBER,PRACTICE_WINNINGS_NUMBER,Y1,Y2
0,98481267304,1,0,M,B,1,0.00000,0.000000,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.449287,101.369863
1,98481267304,2,0,M,B,1,0.13735,0.011550,0.0999,0.0,...,0.0,0.0,1.8,1.0,0.0,0.0,0.0,0.0,4.449287,101.369863
2,98481267304,3,0,M,B,1,0.15835,0.010425,0.0242,0.0,...,0.0,0.0,0.7,0.3,0.0,0.0,0.0,0.0,4.449287,101.369863
3,98481267304,4,0,M,B,1,0.44490,0.035850,0.2398,0.0,...,0.0,0.0,2.8,1.5,0.0,0.0,0.0,0.0,4.449287,101.369863
4,98481267304,5,0,M,B,1,0.00000,0.000000,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.449287,101.369863


In [3]:
X=data.iloc[:,:-2]
y=data.iloc[:,-2:]
X.shape,y.shape,data.shape

((1246837, 23), (1246837, 2), (1246837, 25))

In [4]:
# !pip install category_encoders
import category_encoders as ce

In [5]:
from sklearn.preprocessing import LabelEncoder

# encoder1= LabelEncoder()
# encoder2= LabelEncoder()
# data.iloc[:,3]=encoder1.fit_transform(data.iloc[:,3])
# data.iloc[:,4]=encoder2.fit_transform(data.iloc[:,4])
encoder = ce.BinaryEncoder(cols=["CATEGORY_1","CATEGORY_2"])
X=encoder.fit_transform(X)

/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [6]:
sum(X["CATEGORY_1_0"]),sum(X["CATEGORY_2_0"])
# CATEGORY_1_0 , 2_0 not useful so dropping 'em
X.drop(["CATEGORY_1_0","CATEGORY_2_0"],axis=1,inplace=True)

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.iloc[:,1:], y, test_size=0, random_state=42)

ValueError: test_size=0 should be either positive and smaller than the number of samples 1246837 or a float in the (0, 1) range

In [ ]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [ ]:
X_train[0,:]

In [ ]:
def mse(x,y,modely1,modely2):
    
    y1=modely1.predict(x)
    y2=modely2.predict(x)
    print(y1.shape,y.iloc[:,0].shape)
    mse1=np.sum((y.iloc[:,0]-y1)**2)
    mse2=np.sum((y.iloc[:,1]-y2)**2)
    print(mse1,mse2)
    return (mse1+mse2)/2
    

In [ ]:
regr1 = RandomForestRegressor(max_depth=2,random_state=0)
regr1.fit(X_train, y_train.iloc[:,0])
regr2 = RandomForestRegressor(max_depth=2, random_state=0)
regr2.fit(X_train, y_train.iloc[:,1])

In [ ]:
mse(X_train,y_train,regr1,regr2)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR


In [ ]:
gbreg1 = GradientBoostingRegressor(random_state=0)
gbreg2 = GradientBoostingRegressor(random_state=0)
gbreg1.fit(X_train, y_train.iloc[:,0])
gbreg2.fit(X_train, y_train.iloc[:,1])

In [ ]:
svr1=SVR(C=1.0, epsilon=0.2)
svr2=SVR(C=1.0, epsilon=0.2)
svr1.fit(X_train, y_train.iloc[:,0])
svr2.fit(X_train, y_train.iloc[:,1])

In [ ]:
test=pd.read_csv("/kaggle/input/beyond-analysis/test.csv")

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
test.drop_duplicates(inplace=True)

In [ ]:
test=encoder.transform(test)
test.head()


In [ ]:
test.drop(["CATEGORY_1_0","CATEGORY_2_0"],axis=1,inplace=True)
test.iloc[:,1:]=sc.transform(test.iloc[:,1:])
test.head()

In [ ]:
idx=test.iloc[:,0]
y1=regr1.predict(test.iloc[:,1:])
y2=regr2.predict(test.iloc[:,1:])

In [ ]:
fd=pd.DataFrame({"UNIQUE_IDENTIFIER":idx,"Y1":y1,"Y2":y2})

In [ ]:
fd.groupby(by="UNIQUE_IDENTIFIER").mean().to_csv("sub1.csv")

In [ ]:
import matplotlib.pyplot as plt
importances= regr.feature_importances_
forest_importances = pd.Series(importances, index=data.columns[1:23])

fig, ax = plt.subplots()
forest_importances.plot.bar(ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
ls